In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pylab as plt
import afqstensorutils as atu

In [2]:
%ls

afqstensorutils.py   bound.csv     rescale.py~      surf_scaled.csv~
afqstensorutils.pyc  proto1.ipynb  surf.csv         surf_uv.csv
afqstensorutils.py~  rescale.py    surf_scaled.csv


In [15]:
dataset = tf.contrib.data.CsvDataset(['surf_scaled.csv'],
                                     [tf.float32]*4,
                                     header=True,select_cols=[0,1,2,3])

In [16]:
graph = tf.Graph()
tf.set_random_seed(1234) # Make this notebook deterministic for debugging
with graph.as_default():
    # Input variables of the network
    repeat_dataset = dataset.repeat()
    shuffled_dataset = repeat_dataset.shuffle(buffer_size=100000)
    batched_dataset = shuffled_dataset.batch(2000)
    iterator = batched_dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    stacked_stream = tf.stack(next_element)
    tr_x = tf.transpose(stacked_stream)#
    # The parameters
    N_branch = 3
    N_poly = 3
    N_poly_coeff = int(atu.polyexpand(tf.constant([[1,2]]),N_poly).shape[-1])
    p_V, (We1, ad1,bd1, Wd2) = atu.CatVariable([ (4,2), 
                                             (N_branch*N_poly_coeff,),
                                             (N_branch*N_poly_coeff,), 
                                             (N_branch*N_poly_coeff,4) ],
                                              stddev=0.1)
    # Encoding phase
    # The latent variable
    def encode(x):
        return tf.matmul( x, We1)
    def decode(u):
        branches = tf.split(tf.tile(tr_u, [1,N_branch]),num_or_size_splits=N_branch, axis=1)
        brancha = tf.split(ad1, num_or_size_splits=N_branch, axis=0)
        polys = [ a*atu.polyexpand(x,N_poly) for a,x in zip(brancha,branches) ]
        hd1 = tf.nn.relu( tf.concat(polys,1) + bd1 )
        return tf.matmul(hd1,Wd2)

    tr_u = encode(tr_x)
    tr_xd = decode(tr_u)
    
    # The error is how close they are
    goal_auto = tf.reduce_sum(tf.pow(tr_xd - tr_x,2))
    # We also want the latent space to be contractive:
    # goal_contract = _
    goal = goal_auto
    train_step = tf.train.AdamOptimizer(1e-2).minimize(goal, var_list=[p_V])
    init=tf.global_variables_initializer()
    
    
    i_x = tf.placeholder(tf.float32, shape=(None,4),name='x')
    o_u = encode( i_x )
    # We also want to investigate it
    i_u = tf.placeholder(tf.float32, shape=(None,2),name='u')
    o_xd = decode(i_u)

In [17]:
try:
    sess.close()
except:
    pass
sess = tf.InteractiveSession(graph=graph)

In [18]:
init.run(session=sess)

In [27]:
for i in xrange(1000):
    train_step.run(session=sess)
    if i%100==99: 
        print goal.eval(session=sess)

1.6125185
0.9884883
3.5327249
0.5332583
2.3716073
3.502802
1.1638047
3.1322742
0.88329136
1.4210284


In [23]:
uu,vv = np.meshgrid(np.linspace(-1,1,400), np.linspace(-1,1,400))
uv = np.vstack([uu.ravel(),vv.ravel()])

In [28]:
xduv = o_xd.eval(feed_dict={i_u:uv.T},session=sess)

In [25]:
xduv.shape

(2000, 4)

In [ ]:
from matplotlib import pylab as plt
plt.plot(xduv[:,0],xduv[:,2],',')

In [ ]:
xduv

In [29]:
header="T, p, rho, h"
np.savetxt("surf_uv.csv",xduv,delimiter=", ",header=header,comments="")

In [30]:
We1.eval(session=sess)

array([[-0.19015321, -0.08480992],
       [ 0.48027185, -0.43716195],
       [ 0.97291714,  0.9592274 ],
       [-0.7213462 , -0.61807686]], dtype=float32)